In [1]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 22:15:19 WARN Utils: Your hostname, lnb035652cps resolves to a loopback address: 127.0.1.1; using 192.168.1.172 instead (on interface wlp0s20f3)
25/05/07 22:15:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 22:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 22:15:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 22:15:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [3]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ","
    reader = CSVReader(line_delimiter=",")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/dimensao/dimensao_ibge.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [4]:
df.show()

+--------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|cod_ibge|          municipios|              cod_ra|  reg_administrativa|  reg_metropolitanas|cod_ibge6|             pop2020|       porte_pop2020|             cod_drs|                 drs|
+--------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
| 3500000|Estado sem especi...|Estado sem especi...|Estado sem especi...|Estado sem especi...|   350000|Estado sem especi...|Estado sem especi...|Estado sem especi...|Estado sem especi...|
| 3500105|          Adamantina|                 691|RA de Presidente ...|   Demais municípios|   350010|               33894|Mais de 20 mil at...|                   5|             Marília|
| 3500204|              Adolfo|                 689|RA 

In [5]:
# Instancia a classe de utilitários de qualidade de dados com o DataFrame original
dq = DataQualityUtils(df)

# Encadeia as transformações de qualidade de dados no DataFrame
df_tratado = (
    dq.remove_espacos_em_colunas()  # Remove espaços extras dos nomes das colunas
      .trim_strings()  # Remove espaços em branco no início e fim das strings de todas as colunas
      #.remover_caracteres_especiais(['localidades'])  # (Comentado) Remove caracteres especiais da coluna 'localidades'
      .remover_duplicatas()  # Remove registros duplicados do DataFrame
      .get_df()  # Retorna o DataFrame tratado
)


In [6]:
df_tratado.show()

+--------+--------------------+------+--------------------+--------------------+---------+-------+--------------------+-------+--------------------+
|cod_ibge|          municipios|cod_ra|  reg_administrativa|  reg_metropolitanas|cod_ibge6|pop2020|       porte_pop2020|cod_drs|                 drs|
+--------+--------------------+------+--------------------+--------------------+---------+-------+--------------------+-------+--------------------+
| 3512704|          Corumbataí|   686|      RA de Campinas|    RM de Piracicaba|   351270|   3962|           Até 5 mil|     11|          Piracicaba|
| 3538204|         Pinhalzinho|   686|      RA de Campinas|   Demais municípios|   353820|  14755|Mais de 10 mil at...|      3|            Campinas|
| 3549409|São Joaquim da Barra|   695|        RA de Franca|   Demais municípios|   354940|  50274|Mais de 50 mil at...|      4|              Franca|
| 3528205|           Macedônia|   689|RA de São José do...|   Demais municípios|   352820|   3574|        

In [7]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df_tratado, "../../../layer/prata/dimensao/dimensao_ibge/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")
